In [ ]:
from google.colab import drive


# Mount Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!ls /content/drive/MyDrive/WPI_60K/


all_topics	      faiss_rebuild_checkpoint.json  splade_vectors_v3.jsonl
bm25_index.pkl	      hybrid			     test_topics
csv		      jsonl_docs		     train_topics
data		      Prior_art_qrels.txt	     wpi60k_docnos_cosine.npy
data_sample	      rankings.json		     wpi60k_faiss_cosine.index
eval_checkpoint.json  sample_extracted
extracted	      splade_vectors_v3_ids.jsonl


In [ ]:
# Install faiss if it's not already installed
try:
    import faiss
except ModuleNotFoundError:
    print("Faiss not found. Installing faiss-cpu...")
    !pip install faiss-cpu
    import faiss

In [ ]:
# ✅ Rebuild FAISS Index for Cosine Similarity with Checkpoints (Colab-safe)
import os
import json
import numpy as np
import faiss
from tqdm.notebook import tqdm
from sentence_transformers import SentenceTransformer

# === Paths ===
DRIVE = "/content/drive/MyDrive/WPI_60K"
INPUT_DIR = f"{DRIVE}/extracted/extracted"
INDEX_PATH = f"{DRIVE}/wpi60k_faiss_cosine.index"
DOCNO_PATH = f"{DRIVE}/wpi60k_docnos_cosine.npy"
CHECKPOINT_PATH = f"{DRIVE}/faiss_rebuild_checkpoint.json"

# === Model ===
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# === Checkpoint functions ===
def load_checkpoint():
    if os.path.exists(CHECKPOINT_PATH):
        with open(CHECKPOINT_PATH, 'r') as f:
            return json.load(f)
    return {'processed': [], 'last_file': None}

def save_checkpoint(state):
    with open(CHECKPOINT_PATH, 'w') as f:
        json.dump(state, f)


# === Build FAISS index ===
def build_faiss_cosine_index():
    print("\n🚀 Rebuilding FAISS index with cosine similarity")
    state = load_checkpoint()
    docnos, embeddings = [], []

    # --- Added check for directory existence ---
    if not os.path.exists(INPUT_DIR):
        print(f"❌ Error: Input directory not found: {INPUT_DIR}")
        print("Please ensure Google Drive is mounted correctly and the path is accurate.")
        return
    # --------------------------------------------

    all_files = sorted(f for f in os.listdir(INPUT_DIR) if f.endswith('.json'))
    start_idx = all_files.index(state['last_file']) + 1 if state['last_file'] in all_files else 0

    for file in tqdm(all_files[start_idx:], initial=start_idx, total=len(all_files)):
        try:
            with open(os.path.join(INPUT_DIR, file)) as f:
                doc = json.load(f)

            title = doc.get("title", "")
            abstract = " ".join(doc.get("abstract", "").split()[:100])

            claims_raw = doc.get("claims", "")
            if isinstance(claims_raw, list):
                claims_text = " ".join(claims_raw)
            else:
                claims_text = claims_raw
            claims = " ".join(claims_text.split()[:300])

            description = " ".join(doc.get("description", "").split()[:300])
            text = f"{title} {abstract} {claims} {description}".strip()

            if not text or len(text.split()) < 10:
                continue

            emb = model.encode(text, normalize_embeddings=True)
            docnos.append(doc.get("docno", file.replace(".json", "")))
            embeddings.append(emb.astype('float32'))

            state['processed'].append(file)
            state['last_file'] = file
            if len(state['processed']) % 100 == 0:
                save_checkpoint(state)

        except Exception as e:
            print(f"⚠️ Error processing {file}: {e}")

    save_checkpoint(state)
    print(f"✅ Done encoding {len(docnos)} documents")

    if len(embeddings) == 0:
        print("❌ No valid documents found for indexing. Exiting.")
        return

    matrix = np.stack(embeddings).astype('float32')
    dim = matrix.shape[1]
    index = faiss.IndexFlatIP(dim)
    index.add(matrix)

    faiss.write_index(index, INDEX_PATH)
    np.save(DOCNO_PATH, np.array(docnos))
    print(f"✅ Index and docnos saved. Total vectors: {index.ntotal}, dim={dim}")

# === Run
build_faiss_cosine_index()


🚀 Rebuilding FAISS index with cosine similarity


  0%|          | 0/63923 [00:00<?, ?it/s]

✅ Done encoding 63923 documents
✅ Index and docnos saved. Total vectors: 63923, dim=384
